In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/fer2013.csv')

# Preprocess the data
X = []
y = []

for index, row in df.iterrows():
    pixels = row['pixels'].split(' ')
    X.append(np.array(pixels, dtype='float32'))
    y.append(row['emotion'])

X = np.array(X) / 255.0
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the data
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

# Convert labels to one-hot encoding
le = LabelEncoder()
y_train = to_categorical(le.fit_transform(y_train))
y_test = to_categorical(le.transform(y_test))

# Define the model architecture
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam

def define_model(input_shape=(48, 48, 1), classes=7):
    num_features = 64

    model = Sequential()

    # 1st stage
    model.add(Conv2D(num_features, kernel_size=(3, 3), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Conv2D(num_features, kernel_size=(3, 3)))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.5))

    # 2nd stage
    model.add(Conv2D(num_features, (3, 3), activation='relu'))
    model.add(Conv2D(num_features, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # 3rd stage
    model.add(Conv2D(2 * num_features, kernel_size=(3, 3)))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Conv2D(2 * num_features, kernel_size=(3, 3)))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    # 4th stage
    model.add(Conv2D(2 * num_features, (3, 3), activation='relu'))
    model.add(Conv2D(2 * num_features, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # 5th stage
    model.add(Conv2D(4 * num_features, kernel_size=(3, 3)))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Conv2D(4 * num_features, kernel_size=(3, 3)))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    model.add(Flatten())

    # Fully connected neural networks
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(classes, activation='softmax'))

    return model

# Compiling the model
model = define_model()
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save("/content/drive/MyDrive/ColabNotebooks/model.h5")

Epoch 1/50
449/449 [==============================] - 40s 63ms/step - loss: 1.7744 - accuracy: 0.2780 - val_loss: 1.7035 - val_accuracy: 0.3083
Epoch 2/50
449/449 [==============================] - 27s 60ms/step - loss: 1.4755 - accuracy: 0.4295 - val_loss: 1.7827 - val_accuracy: 0.3632
Epoch 3/50
449/449 [==============================] - 27s 60ms/step - loss: 1.3200 - accuracy: 0.4944 - val_loss: 1.3818 - val_accuracy: 0.4766
Epoch 4/50
449/449 [==============================] - 27s 61ms/step - loss: 1.2308 - accuracy: 0.5352 - val_loss: 1.4232 - val_accuracy: 0.4752
Epoch 5/50
449/449 [==============================] - 26s 58ms/step - loss: 1.1666 - accuracy: 0.5556 - val_loss: 1.2586 - val_accuracy: 0.5241
Epoch 6/50
449/449 [==============================] - 26s 58ms/step - loss: 1.1109 - accuracy: 0.5806 - val_loss: 1.4154 - val_accuracy: 0.4802
Epoch 7/50
449/449 [==============================] - 26s 58ms/step - loss: 1.0572 - accuracy: 0.5998 - val_loss: 1.1642 - val_accuracy:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
